# 00 Load and QC

Load the latest collected CSVs, validate uniqueness, and filter degenerate runs.


# 說明

本 notebook 負責載入彙總 CSV，進行資料品質檢查與基礎過濾，並輸出 QC 表格與過濾後資料。

## 主要輸入
- `outputFiles/analyze/<COLLECT_PREFIX>/collected_stats_*.csv`
- `outputFiles/analyze/<COLLECT_PREFIX>/collected_topk_*.csv`

## 主要輸出
- `tables/qc_summary.csv`
- `tables/qc_basic_checks.csv`
- `tables/qc_missingness.csv`
- `tables/filtered_stats.csv`
- `tables/filtered_out_stats.csv`
- `figures/qc_hist_*.png`

## 合理性檢查建議
- `missing_stats_cols` / `missing_topk_cols` 應為 0
- `runs_missing_in_topk` / `runs_missing_in_stats` 應接近 0
- `filtered_out_stats.csv` 不應佔比過高（視實驗設計）


In [1]:
from pathlib import Path
import os
import re
import pandas as pd
import numpy as np

ANALYZE_DIR = Path('../outputFiles/analyze').resolve()
REPORT_PREFIX = os.environ.get('REPORT_PREFIX', 'analysis_reports')
COLLECT_PREFIX = os.environ.get('COLLECT_PREFIX', REPORT_PREFIX)
REPORT_DIR = (ANALYZE_DIR / REPORT_PREFIX)
COLLECT_DIR = (ANALYZE_DIR / COLLECT_PREFIX)
STATS_CSV = None  # set to a specific file path if needed
TOPK_CSV = None   # set to a specific file path if needed

QC_REQUIRED_STATS_COLS = [
    'run_prefix','search_L','search_W','search_K','search_T',
    'recall_mean',
]
QC_REQUIRED_TOPK_COLS = ['run_prefix']
QC_NUMERIC_CANDIDATES = [
    'build_R','build_L','build_B','build_M',
    'search_K','search_L','search_W','search_T',
    'cache_size','vector_dim','dataset_size','num_queries',
    'recall_mean','latency_p50_us','latency_p90_us','latency_p95_us','latency_p99_us','latency_p999_us',
    'ios_p50','ios_p90','ios_p95','ios_p99',
    'queue_depth_p50','queue_depth_p90','queue_depth_p95','queue_depth_p99',
    'io_us_p50','io_us_p90','io_us_p95','io_us_p99',
    'cpu_us_p50','cpu_us_p90','cpu_us_p95','cpu_us_p99',
    'thread_util_p50','thread_util_p90','thread_util_p95','thread_util_p99',
    'expanded_revisit_ratio','node_counts_top10_share','out_degree_p99',
    'iostat_aqu-sz_mean',
]
QC_RECALL_THRESHOLD = float(os.environ.get('QC_RECALL_THRESHOLD', '0.7'))
QC_RECALL_PCTL = float(os.environ.get('QC_RECALL_PCTL', '0'))
QC_OUTLIER_Z = float(os.environ.get('QC_OUTLIER_Z', '4.0'))
QC_MIN_COUNT = int(os.environ.get('QC_MIN_COUNT', '5'))
QC_EXPECTED_NUM_QUERIES = int(os.environ.get('QC_EXPECTED_NUM_QUERIES', '0'))
QC_PLOT = os.environ.get('QC_PLOT', '1') != '0'

def pick_latest(pattern):
    files = list(COLLECT_DIR.glob(pattern))
    if not files:
        raise FileNotFoundError(f'No files matched: {pattern}')
    # Choose by most recent modification time to ensure "latest" semantics
    return max(files, key=lambda p: p.stat().st_mtime)

def coerce_numeric(df, cols):
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def parse_run_prefix(value):
    if not isinstance(value, str):
        return {}
    pattern = (
        r"_R(?P<build_R>\d+)_L(?P<build_L>\d+)_B(?P<build_B>\d+)_M(?P<build_M>\d+)"
        r"_W(?P<search_W>\d+)_L(?P<search_L>\d+)_K(?P<search_K>\d+)"
        r"_cache(?P<cache_size>\d+)_T(?P<search_T>\d+)$"
    )
    match = re.search(pattern, value)
    if not match:
        return {}
    prefix = value[:match.start()]
    search_id = ''
    index_tag = ''
    if prefix.startswith('S'):
        parts = prefix.split('_', 1)
        if parts and parts[0][1:].isdigit():
            search_id = parts[0][1:]
        if len(parts) > 1:
            index_tag = parts[1]
    else:
        index_tag = prefix
    data = match.groupdict()
    data['search_id'] = search_id
    data['index_tag'] = index_tag
    return data

stats_path = Path(STATS_CSV) if STATS_CSV else pick_latest('collected_stats_*.csv')
topk_path = Path(TOPK_CSV) if TOPK_CSV else pick_latest('collected_topk_*.csv')

print('stats:', stats_path)
print('topk :', topk_path)

stats_df = pd.read_csv(stats_path)
topk_df = pd.read_csv(topk_path)


stats: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/collected_stats_sift01_20260107_195000.csv
topk : /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/collected_topk_sift01_20260107_195000.csv


In [2]:
# Basic integrity checks and schema validation
stats_df = stats_df.copy()
topk_df = topk_df.copy()
stats_df.columns = [str(c).strip() for c in stats_df.columns]
topk_df.columns = [str(c).strip() for c in topk_df.columns]

missing_stats_cols = [c for c in QC_REQUIRED_STATS_COLS if c not in stats_df.columns]
missing_topk_cols = [c for c in QC_REQUIRED_TOPK_COLS if c not in topk_df.columns]
print('missing stats cols:', missing_stats_cols)
print('missing topk cols :', missing_topk_cols)

numeric_cols = [c for c in QC_NUMERIC_CANDIDATES if c in stats_df.columns]
stats_df = coerce_numeric(stats_df, numeric_cols)

print('rows stats:', len(stats_df))
print('rows topk :', len(topk_df))

dup_stats = stats_df['run_prefix'].duplicated().sum() if 'run_prefix' in stats_df.columns else 0
dup_topk = topk_df['run_prefix'].duplicated().sum() if 'run_prefix' in topk_df.columns else 0
print('duplicate run_prefix in stats:', dup_stats)
print('duplicate run_prefix in topk :', dup_topk)

params_cols = [c for c in ['search_L','search_W','search_K','search_T'] if c in stats_df.columns]
if params_cols and 'run_prefix' in stats_df.columns:
    params_per_run = stats_df.groupby('run_prefix')[params_cols].nunique()
    inconsistent = (params_per_run > 1).any(axis=1).sum()
else:
    inconsistent = 0
print('runs with inconsistent search params:', inconsistent)

if 'num_queries' in stats_df.columns:
    num_queries_unique = stats_df['num_queries'].nunique()
    expected_mismatch = 0
    if QC_EXPECTED_NUM_QUERIES:
        expected_mismatch = (stats_df['num_queries'] != QC_EXPECTED_NUM_QUERIES).sum()
else:
    num_queries_unique = 0
    expected_mismatch = 0
print('num_queries unique values:', num_queries_unique)
if QC_EXPECTED_NUM_QUERIES:
    print('num_queries mismatching expected:', expected_mismatch)

missing_df = pd.DataFrame({
    'column': stats_df.columns,
    'missing_count': stats_df.isna().sum().values,
    'missing_rate': stats_df.isna().mean().values,
}).sort_values('missing_rate', ascending=False)

if 'run_prefix' in stats_df.columns and 'run_prefix' in topk_df.columns:
    stats_runs = set(stats_df['run_prefix'])
    topk_runs = set(topk_df['run_prefix'])
    missing_in_topk = sorted(stats_runs - topk_runs)
    missing_in_stats = sorted(topk_runs - stats_runs)
else:
    missing_in_topk = []
    missing_in_stats = []

qc_basic_df = pd.DataFrame([
    {
        'total_stats_rows': len(stats_df),
        'total_topk_rows': len(topk_df),
        'missing_stats_cols': len(missing_stats_cols),
        'missing_topk_cols': len(missing_topk_cols),
        'duplicate_stats_run_prefix': int(dup_stats),
        'duplicate_topk_run_prefix': int(dup_topk),
        'runs_with_inconsistent_search_params': int(inconsistent),
        'num_queries_unique_values': int(num_queries_unique),
        'num_queries_mismatch_expected': int(expected_mismatch),
        'runs_missing_in_topk': len(missing_in_topk),
        'runs_missing_in_stats': len(missing_in_stats),
    }
])
qc_basic_df


missing stats cols: []
missing topk cols : []
rows stats: 3150
rows topk : 3150
duplicate run_prefix in stats: 0
duplicate run_prefix in topk : 0
runs with inconsistent search params: 0
num_queries unique values: 1


,total_stats_rows,total_topk_rows,missing_stats_cols,missing_topk_cols,duplicate_stats_run_prefix,duplicate_topk_run_prefix,runs_with_inconsistent_search_params,num_queries_unique_values,num_queries_mismatch_expected,runs_missing_in_topk,runs_missing_in_stats
0,3150,3150,0,0,0,0,0,1,0,0,0


In [3]:
# Filtering rules with explicit flags
flags = pd.DataFrame(index=stats_df.index)

if 'search_L' in stats_df.columns and 'search_K' in stats_df.columns:
    flags['flag_search_L_lt_K'] = stats_df['search_L'] < stats_df['search_K']
if 'search_L' in stats_df.columns and 'search_W' in stats_df.columns:
    flags['flag_search_L_lt_2W'] = stats_df['search_L'] < 2 * stats_df['search_W']

recall_threshold = QC_RECALL_THRESHOLD
if 'recall_mean' in stats_df.columns and QC_RECALL_PCTL > 0:
    recall_threshold = max(recall_threshold, stats_df['recall_mean'].quantile(QC_RECALL_PCTL))
if 'recall_mean' in stats_df.columns:
    flags['flag_low_recall'] = stats_df['recall_mean'] < recall_threshold
    flags['flag_recall_out_of_range'] = (stats_df['recall_mean'] < 0) | (stats_df['recall_mean'] > 1)

if 'latency_p99_us' in stats_df.columns:
    flags['flag_latency_p99_nonpositive'] = stats_df['latency_p99_us'] <= 0
if 'latency_p999_us' in stats_df.columns:
    flags['flag_latency_p999_nonpositive'] = stats_df['latency_p999_us'] <= 0

for col in ['ios_p99','queue_depth_p99','io_us_p99','cpu_us_p99']:
    if col in stats_df.columns:
        flags[f'flag_{col}_negative'] = stats_df[col] < 0
if 'thread_util_p99' in stats_df.columns:
    flags['flag_thread_util_out_of_range'] = (stats_df['thread_util_p99'] < 0) | (stats_df['thread_util_p99'] > 1.1)
if 'vector_dim' in stats_df.columns:
    flags['flag_vector_dim_nonpositive'] = stats_df['vector_dim'] <= 0
if 'dataset_size' in stats_df.columns:
    flags['flag_dataset_size_nonpositive'] = stats_df['dataset_size'] <= 0
if 'num_queries' in stats_df.columns:
    flags['flag_num_queries_nonpositive'] = stats_df['num_queries'] <= 0

flags = flags.fillna(False)

for col in flags.columns:
    stats_df[col] = flags[col]

exclude_any = flags.any(axis=1)
filtered = stats_df[~exclude_any].copy()
filtered_out = stats_df[exclude_any].copy()
if not filtered_out.empty:
    def build_reason(row):
        return ','.join([c for c in flags.columns if row.get(c, False)])
    filtered_out['exclude_reason'] = filtered_out.apply(build_reason, axis=1)

print('filtered rows:', len(filtered))

qc_summary = {
    'total_rows': len(stats_df),
    'filtered_rows': len(filtered),
    'excluded_rows': int(exclude_any.sum()),
    'low_recall_threshold': recall_threshold,
}
for col in flags.columns:
    qc_summary[col] = int(flags[col].sum())

qc_df = pd.DataFrame([qc_summary])
qc_df


filtered rows: 2955


,total_rows,filtered_rows,excluded_rows,low_recall_threshold,flag_search_L_lt_K,flag_search_L_lt_2W,flag_low_recall,flag_recall_out_of_range,flag_latency_p99_nonpositive,flag_latency_p999_nonpositive,flag_ios_p99_negative,flag_queue_depth_p99_negative,flag_io_us_p99_negative,flag_cpu_us_p99_negative,flag_thread_util_out_of_range,flag_vector_dim_nonpositive,flag_dataset_size_nonpositive,flag_num_queries_nonpositive
0,3150,2955,195,0.7,0,0,195,0,0,0,0,0,0,0,0,0,0,0


In [4]:
# Save QC outputs (basic)
out_tables = (REPORT_DIR / 'tables')
out_tables.mkdir(parents=True, exist_ok=True)

qc_outputs = {
    'qc_summary.csv': qc_df,
    'qc_basic_checks.csv': qc_basic_df,
    'qc_missingness.csv': missing_df,
    'filtered_stats.csv': filtered,
    'filtered_out_stats.csv': filtered_out,
}

for name, df in qc_outputs.items():
    if df is None:
        continue
    df.to_csv(out_tables / name, index=False)
    print('Saved:', out_tables / name)


Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_summary.csv
Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_basic_checks.csv
Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_missingness.csv


Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/filtered_stats.csv


Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/filtered_out_stats.csv


In [5]:
# Parse run_prefix and validate parameter consistency
runprefix_parsed_df = pd.DataFrame()
runprefix_mismatch_df = pd.DataFrame()
runprefix_summary_df = pd.DataFrame()

if 'run_prefix' in stats_df.columns:
    parsed = stats_df['run_prefix'].apply(parse_run_prefix)
    parsed_df = pd.DataFrame(list(parsed))
    if not parsed_df.empty:
        parsed_df = parsed_df.add_prefix('parsed_')
        stats_df = pd.concat([stats_df, parsed_df], axis=1)
        parse_ok = parsed_df.notna().any(axis=1)
        runprefix_summary_df = pd.DataFrame([
            {
                'total_rows': len(stats_df),
                'parse_ok_rows': int(parse_ok.sum()),
                'parse_fail_rows': int((~parse_ok).sum()),
            }
        ])
        mismatch_rows = []
        for col in parsed_df.columns:
            raw = col.replace('parsed_', '')
            if raw in stats_df.columns:
                left_series = stats_df[raw]
                right_series = parsed_df[col]
                # Prefer numeric comparison when both sides are numeric
                left_num = pd.to_numeric(left_series, errors='coerce')
                right_num = pd.to_numeric(right_series, errors='coerce')
                both_numeric = left_num.notna() & right_num.notna()
                mism_num = both_numeric & (left_num != right_num)
                # Fallback to trimmed string comparison; ignore NaN on right
                left_str = left_series.astype(str).str.strip()
                right_str = right_series.astype(str).str.strip()
                mism_str = (~both_numeric) & (right_str != 'nan') & (left_str != right_str)
                mism = mism_num | mism_str
                for idx in stats_df.index[mism]:
                    mismatch_rows.append({
                        'run_prefix': stats_df.loc[idx, 'run_prefix'],
                        'field': raw,
                        'stats_value': stats_df.loc[idx, raw],
                        'parsed_value': parsed_df.loc[idx, col],
                    })
        runprefix_mismatch_df = pd.DataFrame(mismatch_rows)
        runprefix_parsed_df = pd.concat([stats_df[['run_prefix']], parsed_df], axis=1)

runprefix_summary_df


,total_rows,parse_ok_rows,parse_fail_rows
0,3150,3150,0


In [6]:
# Outlier detection and descriptive stats
outlier_summary_df = pd.DataFrame()
outlier_samples_df = pd.DataFrame()
basic_stats_df = pd.DataFrame()

numeric_cols = [c for c in stats_df.columns if pd.api.types.is_numeric_dtype(stats_df[c])]
if numeric_cols:
    stats_numeric = stats_df[numeric_cols]
    basic_stats_df = stats_numeric.describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99]).T
    basic_stats_df['missing_rate'] = stats_numeric.isna().mean()
    basic_stats_df['zero_count'] = (stats_numeric == 0).sum()

    summary_rows = []
    sample_rows = []
    for col in numeric_cols:
        series = stats_numeric[col].dropna()
        if series.nunique() < 2:
            continue
        median = series.median()
        mad = np.median(np.abs(series - median))
        if mad <= 0:
            continue
        z = 0.6745 * (series - median) / mad
        mask = z.abs() > QC_OUTLIER_Z
        count = int(mask.sum())
        if count:
            summary_rows.append({
                'column': col,
                'outlier_count': count,
                'outlier_rate': count / max(len(series), 1),
                'median': median,
                'mad': mad,
            })
            for idx in series.index[mask]:
                sample_rows.append({
                    'run_prefix': stats_df.loc[idx, 'run_prefix'] if 'run_prefix' in stats_df.columns else idx,
                    'column': col,
                    'value': stats_df.loc[idx, col],
                    'robust_z': z.loc[idx],
                })

    outlier_summary_df = pd.DataFrame(summary_rows).sort_values('outlier_count', ascending=False)
    outlier_samples_df = pd.DataFrame(sample_rows)

outlier_summary_df.head(10)


,column,outlier_count,outlier_rate,median,mad
127,iostat_w/s_max,1373,0.435873,6.750000,0.770000
133,iostat_wkB/s_max,1172,0.372063,671.220000,84.330000
121,iostat_rareq-sz_max,1097,0.348254,4.040000,0.020000
129,iostat_w_await_max,955,0.303175,3.500000,0.070000
107,node_counts_top1_share,930,0.295238,0.009509,0.002453
109,iostat_%rrqm_max,926,0.293968,1.005000,0.625000
40,io_us_max,924,0.293333,4406.100100,2437.202100
27,latency_max_us,924,0.293333,4798.319350,2672.486800
113,iostat_%wrqm_max,915,0.290476,83.330000,7.580000
119,iostat_r_await_max,868,0.275556,0.150000,0.010000


In [7]:
# Track missing run_prefix across stats/topk
missing_in_topk_df = pd.DataFrame({'run_prefix': missing_in_topk})
missing_in_stats_df = pd.DataFrame({'run_prefix': missing_in_stats})

missing_in_topk_df.head(10)


,run_prefix


In [8]:
# Optional QC plots
if QC_PLOT:
    import matplotlib.pyplot as plt
    fig_dir = (REPORT_DIR / 'figures')
    fig_dir.mkdir(parents=True, exist_ok=True)

    plot_cols = [
        'recall_mean',
        'latency_p99_us',
        'latency_p999_us',
        'io_us_p99',
        'queue_depth_p99',
        'expanded_revisit_ratio',
        'node_counts_top10_share',
    ]
    plot_cols = [c for c in plot_cols if c in stats_df.columns]
    for col in plot_cols:
        series = stats_df[col].dropna()
        if series.empty:
            continue
        plt.figure(figsize=(6, 4))
        plt.hist(series, bins=50, color='#4c78a8', alpha=0.8)
        plt.title(col)
        plt.tight_layout()
        plt.savefig(fig_dir / f'qc_hist_{col}.png', dpi=150)
        plt.close()

    if 'recall_mean' in stats_df.columns and 'latency_p99_us' in stats_df.columns:
        plt.figure(figsize=(6, 4))
        plt.scatter(stats_df['recall_mean'], stats_df['latency_p99_us'], s=10, alpha=0.6)
        plt.xlabel('recall_mean')
        plt.ylabel('latency_p99_us')
        plt.tight_layout()
        plt.savefig(fig_dir / 'qc_scatter_recall_latency_p99.png', dpi=150)
        plt.close()


In [9]:
# Save additional QC outputs
extra_outputs = {
    'qc_runprefix_parsed.csv': runprefix_parsed_df,
    'qc_runprefix_mismatch.csv': runprefix_mismatch_df,
    'qc_runprefix_summary.csv': runprefix_summary_df,
    'qc_basic_stats.csv': basic_stats_df,
    'qc_outlier_summary.csv': outlier_summary_df,
    'qc_outlier_samples.csv': outlier_samples_df,
    'qc_missing_in_topk.csv': missing_in_topk_df,
    'qc_missing_in_stats.csv': missing_in_stats_df,
}

for name, df in extra_outputs.items():
    if df is None:
        continue
    df.to_csv(out_tables / name, index=False)
    print('Saved:', out_tables / name)


Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_runprefix_parsed.csv
Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_runprefix_mismatch.csv
Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_runprefix_summary.csv
Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_basic_stats.csv
Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_outlier_summary.csv
Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_outlier_samples.csv
Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_missing_in_topk.csv
Saved: /home/gt/research/DiskANN/scripts/paramAnalysis/gridSearch/outputFiles/analyze/sift01/tables/qc_missing_in_stats.csv
